In [1]:
import pandas as pd
from openpyxl import Workbook

In [2]:
def ABC_segmentation(cum_sum):
    '''Функция для группировки ABC 50-30-20'''
    if cum_sum <= 0.5:
        return 'A'
    elif cum_sum <= 0.8:
        return 'B'
    else:
        return 'C'

In [3]:
data = pd.read_excel('http://file-st10.karelia.ru/kftgng/77a1cac290ab62504c3afbe1f0f8fbaa/3d20bda4a5b738dfe16cedd22028301e/240221_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20ABC.xlsx?force', sheet_name='Для сторонних источников', header=0, index_col=False)
data_full = data.copy(deep=True) #создание двух датафреймов, из файла - один по каналам продаж, другой - по компании в целом

In [4]:
data['rank'] = data.groupby(['Канал продаж'])['Общее количество'].rank('dense', ascending=False)
data = data.sort_values(by=['Канал продаж', 'rank'], ascending=False)
# создание столбца rank по группам опт-розница, с последующей сортировкой ранга по убыванию

In [6]:
data['fraction'] = data['Общее количество']/data.groupby(['Канал продаж'])['Общее количество'].transform('sum')
data['cum_sum'] = data.groupby(['Канал продаж'])['fraction'].transform('cumsum')
# создание столбцов "доля", "накопительная сумма доли" для каждой группы канала продаж

In [7]:
data['group'] = data['cum_sum'].apply(ABC_segmentation)
# применение функции группировки ABC

In [11]:
data = data.rename(columns={'Канал продаж': 'sales_channel', 'Номенклатура': 'SKU', 'Общее количество': 'total'})
# переименование столбцов для удобства

In [12]:
data

,sales_channel,SKU,total,rank,fraction,cum_sum,group
19,розница,a1,1202,22.0,0.022183,0.022183,A
20,розница,a2,1407,21.0,0.025966,0.048149,A
21,розница,a3,1414,20.0,0.026095,0.074244,A
23,розница,a5,1556,19.0,0.028716,0.102960,A
22,розница,a4,1561,18.0,0.028808,0.131768,A
24,розница,a6,1731,17.0,0.031946,0.163714,A
25,розница,a8,1801,16.0,0.033237,0.196951,A
26,розница,a9,1819,15.0,0.033570,0.230521,A
28,розница,a11,2021,14.0,0.037297,0.267818,A
27,розница,a10,2153,13.0,0.039734,0.307552,A


In [9]:
result_data = data.groupby(['sales_channel', 'group']).aggregate({'SKU': 'count', 'total': 'sum'})
# получение сводной таблицы

In [10]:
result_data

SKU  total
sales_channel group            
опт           A       11  24663
              B        5  16805
              C        3  12604
розница       A       14  26317
              B        5  15794
              C        3  12075

In [13]:
data_full.drop('Канал продаж', axis=1, inplace=True)
# удаление каналов продаж для расчета в целом для компании

In [14]:
data_full = data_full.groupby(['Номенклатура'])['Общее количество'].sum().reset_index()
# группировка по SKU

In [15]:
data_full['fraction'] = data_full['Общее количество']/data_full['Общее количество'].sum()
data_full = data_full.sort_values(by='fraction', ascending=False)
# создание столбца доли и сортировка по убыванию доли

In [19]:
data_full['cum_sum'] = data_full['fraction'].cumsum()
data_full['group'] = data_full['cum_sum'].apply(ABC_segmentation)
# создание столбцов "накопительная сумма доли" и "группа" c применением функции

In [20]:
data_full

,Номенклатура,Общее количество,fraction,cum_sum,group
17,a25,8536,0.078849,0.078849,A
15,a23,7921,0.073168,0.152016,A
14,a22,7863,0.072632,0.224649,A
9,a18,6599,0.060956,0.285605,A
10,a19,6522,0.060245,0.345850,A
6,a15,5713,0.052772,0.398622,A
8,a17,5664,0.052319,0.450941,A
4,a13,5088,0.046999,0.497940,A
1,a10,4713,0.043535,0.541475,B
2,a11,4550,0.042029,0.583504,B


In [17]:
result_data_full = data_full.groupby('group').aggregate({'Номенклатура': 'count', 'Общее количество': 'sum'})
result_data_full = result_data_full.rename(columns={'Номенклатура': 'SKU', 'Общее количество': 'total'})
# получение сводной таблицы и переименование столбцов

In [18]:
result_data_full

,SKU,total
group,,
A,8,53906
B,8,31711
C,9,22641
